Este script tiene como objetivo aplicar varios modelos de machine learning para predecir la mortalidad a partir de variables climáticas en diferentes provincias. Para ello, se emplean varios algoritmos, entre ellos Poisson, Random Forest, Lasso, Árbol de Decisión, Redes Neuronales y ARIMA. El análisis se realiza para cada provincia individualmente y los resultados se evalúan mediante dos métricas principales: Mean Squared Error (MSE) y Coeficiente de Determinación (R²).

In [ ]:
import pandas as pd
from sklearn.linear_model import PoissonRegressor, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import os
import warnings  # Para suprimir las advertencias

# Suprimir advertencias que no afectan el rendimiento
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=ValueWarning)

# Cargar los datos desde la carpeta data
ruta_csv = os.path.join('data', 'dataset_modificado.csv')
df = pd.read_csv(ruta_csv)

# Definir los modelos a utilizar
modelos = {
    "Poisson": PoissonRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Lasso": Lasso(),
    "Decision Tree": DecisionTreeRegressor(),
    "Red Neuronal": MLPRegressor()
}

# Crear una lista para almacenar los resultados
resultados = []

# Loop para procesar cada provincia
for provincia in df['provincia'].unique():
    print(f"Procesando provincia: {provincia}")
    
    # Filtrar datos específicos de la provincia
    df_provincia = df[df['provincia'] == provincia]
    
    # Definir las variables independientes (X) y dependiente (y)
    X = df_provincia[['temp_min_max', 'temp_max_max', 'humedad_max',
                      'dias_consecutivos_tmin_22', 'dias_consecutivos_tmin_25',
                      'dias_consecutivos_tmin_28', 'dias_consecutivos_tmax_35',
                      'dias_consecutivos_tmax_38', 'dias_consecutivos_tmax_41',
                      'dias_consecutivos_humedad_70', 'dias_consecutivos_humedad_80',
                      'dias_consecutivos_humedad_90', 'dias']]
    y = df_provincia['mortalidad']
    
    # Dividir los datos en entrenamiento (80%) y prueba (20%)
    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]
    
    # Entrenar y evaluar cada modelo de machine learning
    for nombre_modelo, modelo in modelos.items():
        # Entrenar el modelo con los datos de entrenamiento
        modelo.fit(X_train, y_train)
        
        # Hacer predicciones con los datos de prueba
        y_pred = modelo.predict(X_test)
        
        # Calcular métricas de rendimiento
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Almacenar los resultados del modelo
        resultados.append({
            'provincia': provincia,
            'modelo': nombre_modelo,
            'MSE': mse,
            'R²': r2
        })

    # Modelo ARIMA para la serie temporal de mortalidad
    arima_model = ARIMA(y_train, order=(5, 1, 0))
    arima_model_fit = arima_model.fit()
    y_pred_arima = arima_model_fit.forecast(steps=len(y_test))
    
    # Calcular métricas de rendimiento para ARIMA
    mse_arima = mean_squared_error(y_test, y_pred_arima)
    r2_arima = r2_score(y_test, y_pred_arima)
    
    # Almacenar los resultados de ARIMA
    resultados.append({
        'provincia': provincia,
        'modelo': 'ARIMA',
        'MSE': mse_arima,
        'R²': r2_arima
    })

# Convertir los resultados a un DataFrame para análisis
df_resultados = pd.DataFrame(resultados)

# Guardar los resultados en un archivo CSV en la carpeta data
ruta_resultados = os.path.join('data', 'resultados_modelos_provincias.csv')
df_resultados.to_csv(ruta_resultados, index=False)

# Notificar al usuario que los resultados han sido guardados
print(f"Resultados guardados en {ruta_resultados}")
